# Water Quality Prediction: Benchmark Notebook 

## Challenge Overview

Welcome to the EY AI & Data Challenge 2026!  
The objective of this challenge is to build a robust **machine learning model** capable of predicting water quality across various river locations in South Africa. In addition to accurate predictions, the model should also identify and emphasize the key factors that significantly influence water quality.

Participants will be provided with a dataset containing three water quality parameters — **Total Alkalinity**, **Electrical Conductance**, and **Dissolved Reactive Phosphorus** — collected between 2011 and 2015 from approximately 200 river locations across South Africa. Each data point includes the geographic coordinates (latitude and longitude) of the sampling site, the date of collection, and the corresponding water quality measurements.

Using this dataset, participants are expected to build a machine learning model to predict water quality parameters for a separate validation dataset, which includes locations from different regions not present in the training data. The challenge also encourages participants to explore feature importance and provide insights into the factors most strongly associated with variations in water quality.

This challenge is designed for participants with varying levels of experience in data science, remote sensing, and environmental analytics. It offers a valuable opportunity to apply machine learning techniques to real-world environmental data and contribute to advancing water quality monitoring using artificial intelligence.

**About the Notebook:**  

In this notebook, we demonstrate a basic workflow that serves as a foundation for the challenge. The model has been developed to predict **water quality parameters** using features derived from the **Landsat** and **TerraClimate** datasets. Specifically, four spectral bands — **SWIR22** (Shortwave Infrared 2), **NIR** (Near Infrared), **Green**, and **SWIR16** (Shortwave Infrared 1) — were utilized from Landsat, along with derived spectral indices such as **NDMI** (Normalized Difference Moisture Index) and **MNDWI** (Modified Normalized Difference Water Index). In addition, the **PET** (Potential Evapotranspiration) variable was incorporated from the **TerraClimate** dataset to account for climatic influences on water quality.

The dataset spans a five-year period from **2011 to 2015**. Using **API-based data extraction** methods, both Landsat and TerraClimate features were retrieved directly from the [Microsoft Planetary Computer portal](https://planetarycomputer.microsoft.com).

These combined spectral, index-based, and climatic features were used as predictors in a regression model to estimate three key water quality parameters: **Total Alkalinity (TA)**, **Electrical Conductance (EC)**, and **Dissolved Reactive Phosphorus (DRP)**.

Please note that this notebook serves only as a starting point. Several assumptions were made during the data extraction and model development process, which you may find opportunities to improve upon. Participants are encouraged to explore additional features, enhance preprocessing techniques, or experiment with different regression algorithms to optimize predictive performance.

## Load In Dependencies
The following code installs the required Python libraries (found in the requirements.txt file) in the Snowflake environment to allow successful execution of the remaining notebook code. After running this code for the first time, it is required to “restart” the kernal so the Python libraries are available in the environment. This is done by selecting the “Connected” menu above the notebook (next to “Run all”) and selecting the “restart kernal” link. Subsequent runs of the notebook do not require this “restart” process.

In [ ]:
!pip install uv
!uv pip install  -r requirements.txt

In [1]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation and analysis
import numpy as np
import pandas as pd
from IPython.display import display

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

# Geospatial raster data handling with CRS support
import rioxarray as rxr

# Raster operations and spatial windowing
import rasterio
from rasterio.windows import Window

# Feature preprocessing and data splitting
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os 

## Response Variable

Before building the model, we first load the **water quality training dataset**. The curated dataset contains samples collected from various monitoring stations across the study region. Each record includes the geographical coordinates (Latitude and Longitude), the sample collection date, and the corresponding **measured values** for the three key water quality parameters — **Total Alkalinity (TA)**, **Electrical Conductance (EC)**, and **Dissolved Reactive Phosphorus (DRP)**.

In [ ]:
Water_Quality_df = pd.read_csv("water_quality_training_dataset.csv")
display(Water_Quality_df.head(5))

MY CODE:
READ DATA FROM SNOWFLAKE TABLES

In [ ]:
%%sql -r dataframe_1
USE DATABASE OPTIMIZING_CLEAN_WATER_SUPPLY

In [ ]:
train_df = session.table('FEATURES.FEATURE_MART_TRAIN_IMPUTED').to_pandas()
display(train_df.head())

In [ ]:
train_df.columns

## Predictor Variables

Now that we have our water quality dataset, the next step is to gather the predictor variables from the **Landsat** and **TerraClimate** datasets. In this notebook, we demonstrate how to **load previously extracted satellite and climate data** from separate files, rather than performing the extraction directly, which allows for a smoother and faster experience. Participants can refer to the dedicated extraction notebooks—one for Landsat and another for TerraClimate—to understand how the data was retrieved and processed, and they can also generate their own output CSV files if needed. Using these pre-extracted CSV files, this notebook focuses on loading the predictor features and running the subsequent analysis and model training efficiently.

For more detailed guidance on the original data extraction process, you can review the Landsat and TerraClimate example notebooks available on the Planetary Computer portal:

- [Landsat-c2-l2 - Example-Notebook](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2#Example-Notebook)  
- [Terraclimate - Example-Notebook](https://planetarycomputer.microsoft.com/dataset/terraclimate#Example-Notebook)

We have used selected spectral bands — **SWIR22** (Shortwave Infrared 2), **NIR** (Near Infrared), **Green**, and **SWIR16** (Shortwave Infrared 1) — and computed key spectral indices such as **NDMI** (Normalized Difference Moisture Index) and **MNDWI** (Modified Normalized Difference Water Index). These features capture surface moisture, vegetation, and water content characteristics that influence water quality variability.

In addition to Landsat features, we also incorporated the **Potential Evapotranspiration (PET)** variable from the **TerraClimate** dataset, which provides high-resolution global climate data. The PET feature captures the atmospheric demand for moisture, representing climatic conditions such as temperature, humidity, and radiation that influence surface water evaporation and thus affect water quality parameters.

The predictor features include:

- **SWIR22** – Sensitive to surface moisture and turbidity variations in water bodies.  
- **NIR** – Helps in identifying vegetation and suspended matter in water.  
- **Green** – Useful for detecting water color and surface reflectance changes.  
- **SWIR16** – Provides information on surface dryness and sediment concentration.  
- **NDMI** – Derived from NIR and SWIR16, indicates moisture and vegetation–water interaction.  
- **MNDWI** – Derived from Green and SWIR22, effective for distinguishing open water areas and reducing built-up noise.  
- **PET** – Extracted from the TerraClimate dataset, represents potential evapotranspiration influencing hydrological and water quality dynamics.

### **Tip 1**

Participants are encouraged to experiment with different combinations of **Landsat** bands or even include data from other public satellite data sources. By creating mathematical combinations of bands, you can derive various spectral indices that capture surface and environmental characteristics.

### Loading Pre-Extracted Landsat Data

In this notebook, we **load previously extracted Landsat data** from CSV files generated in a separate extraction notebook. This approach ensures a smoother and faster workflow, allowing participants to focus on data analysis and model development without waiting for time-consuming data retrieval.

Participants are expected to generate their own data extraction CSV files by running the dedicated Landsat extraction notebook. These CSV files can then be used here to smoothly run this benchmark notebook. Participants can refer to the extraction notebook to understand the API-based process, including how individual bands and indices like **NDMI** were computed. Using these pre-extracted CSV files simplifies preprocessing and is ideal for large-scale environmental and water quality analysis.

### **Tip 2**

In the data extraction process (performed in the dedicated extraction notebooks), a 100 m focal buffer was applied around each sampling location rather than using a single point. Participants may explore creating different focal buffers around the locations (e.g., 50 m, 150 m, etc.) during extraction. For example, if a 50 m buffer was used for “Band 2”, the extracted CSV values would reflect the average of Band 2 within 50 meters of each location. This approach can help reduce errors associated with spatial autocorrelation.

In [ ]:
landsat_train_features = pd.read_csv("landsat_features_training.csv")
display(landsat_train_features.head(5))

In [ ]:
# If NDMI and MNDWI columns are of type object, convert them to float
landsat_train_features['NDMI'] = landsat_train_features['NDMI'].astype(float)
landsat_train_features['MNDWI'] = landsat_train_features['MNDWI'].astype(float)

### Loading Pre-Extracted TerraClimate Data

In this notebook, we **load previously extracted TerraClimate data** from CSV files generated in a dedicated extraction notebook. This approach ensures a smoother and faster workflow, allowing participants to focus on data analysis and model development without waiting for time-consuming data retrieval.

Participants are expected to generate their own data extraction CSV files by running the dedicated TerraClimate extraction notebook. These CSV files can then be used here to smoothly run this benchmark notebook. Participants can refer to the extraction notebook to understand the API-based process, including how climate variables such as **Potential Evapotranspiration (PET)** were extracted. Using these pre-extracted CSV files ensures consistent, automated retrieval of high-resolution climate data that can be easily integrated with satellite-derived features for comprehensive environmental and hydrological analysis.

In [ ]:
Terraclimate_df = pd.read_csv("terraclimate_features_training.csv")
display(Terraclimate_df.head(5))

## Joining the Predictor Variables and Response Variables

Now that we have extracted our predictor variables, we need to join them with the response variables. We use the **combine_two_datasets** function to merge the predictor variables and response variables. The **concat** function from pandas is particularly useful for this step.

In [5]:
# Combine two datasets vertically (along columns) using pandas concat function.
def combine_two_datasets(dataset1,dataset2,dataset3):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2,dataset3], axis=1)
    data = data.loc[:, ~data.columns.duplicated()]
    return data

In [6]:
# Combining ground data and final data into a single dataset.
wq_data = combine_two_datasets(Water_Quality_df, landsat_train_features, Terraclimate_df)
display(wq_data.head(5))

### Handling Missing Values

Before model training, missing values in the dataset were carefully handled to ensure data consistency and prevent model bias. Numerical columns were imputed using their median values, maintaining the overall data distribution while minimizing the impact of outliers.

In [7]:
wq_data = wq_data.fillna(wq_data.median(numeric_only=True))
wq_data.isna().sum()

## Model Building

Now let us select the columns required for our model-building exercise. We will consider only **SWIR22**, **NDMI**, and **MNDWI** from the Landsat data, and **PET** from the TerraClimate dataset as our predictor variables. It does not make sense to use latitude and longitude as predictor variables, as they do not have any direct impact on predicting the water quality parameters.

In [ ]:
# Retaining only the columns for swir22, NDMI, MNDWI, pet, Total Alkalinity, Electrical Conductance and Dissolved Reactive Phosphorus Index in the dataset.
wq_data = wq_data[['swir22','NDMI','MNDWI','pet', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']]

MY CODE: REPLACE FEATURES AND TARGET VARIABLES

In [ ]:
TARGETS = ["TOTAL_ALKALINITY", "ELECTRICAL_CONDUCTANCE", "DISSOLVED_REACTIVE_PHOSPHORUS"]

FEATURES = [
    "SWIR22", "NDMI", "MNDWI", "PET",
    "SIN_DOY", "COS_DOY", "NIR", "GREEN", "SWIR16"
]

X = train_df[FEATURES]
y_TA  = train_df["TOTAL_ALKALINITY"]
y_EC  = train_df["ELECTRICAL_CONDUCTANCE"]
y_DRP = train_df["DISSOLVED_REACTIVE_PHOSPHORUS"]

### **Tip 3**

We are developing individual models for each water quality parameter using a common set of features: **SWIR22**, **NDMI**, **MNDWI**, and **PET**. However, participants are encouraged to experiment with different feature combinations to build more robust machine learning models.

## Helper Functions

### Train and Test Split
We will now split the data into 70% training data and 30% test data. Scikit-learn (sklearn) is a robust library for machine learning in Python. The `model_selection` module in scikit-learn provides the `train_test_split` function, which can be used for this purpose.

### Feature Scaling
Before initiating model training, we may need to perform various data preprocessing steps. Here, we demonstrate scaling of the variables **SWIR22**, **NDMI**, **MNDWI**, and **PET** using StandardScaler.

Feature scaling is an essential preprocessing step for numerical features. Many machine learning algorithms—such as gradient descent methods, KNN, and linear or logistic regression—require scaling to achieve optimal performance. Scikit-learn provides several scaling utilities. In this notebook, we use **StandardScaler**, which transforms the data so that each feature has a mean of 0 and a standard deviation of 1.

### Model Training
Now that we have the data in a format suitable for machine learning, we can begin training our models. In this demonstration notebook, we build three separate regression models—one for each target water quality parameter: **Total Alkalinity**, **Electrical Conductance**, and **Dissolved Reactive Phosphorus**. Each model is trained independently to capture the unique relationships between the satellite-derived features and each water quality parameter.

We use the **Random Forest Regressor** from the scikit-learn library for model training. Scikit-learn offers a wide range of regression algorithms, along with powerful parameter tuning and customization options.

For model training, the predictor variables (e.g., SWIR22, NDMI, MNDWI, and PET) are stored in an array `X`, and the response variable (one of the water quality parameters) is stored in an array `Y`. Note that the response variable should not be included in `X`. Also, latitude, longitude, and sample date are excluded from the predictor variables since they serve only as spatial and temporal references.

### Model Evaluation
After training the models for the three water quality parameters, the next step is to evaluate their performance. Each regression model—Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus—is assessed using:

- **R² Score**: Measures how well the model explains the variance in the observed values.  
- **RMSE (Root Mean Square Error)**: Quantifies the average magnitude of prediction errors.

Together, these metrics help determine how effectively each model captures variations in water quality across locations and sampling dates. Scikit-learn provides built-in functions to compute both metrics. Participants may also explore additional evaluation techniques or custom metrics to enhance model assessment.

### **Tip 4**

There are many data preprocessing methods available that may help improve model performance. Participants are encouraged to explore various preprocessing techniques as well as different machine learning algorithms to build a more robust model.

In [9]:
def split_data(X, y, test_size=0.3, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

def train_model(X_train_scaled, y_train):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    return model

def evaluate_model(model, X_scaled, y_true, dataset_name="Test"):
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"\n{dataset_name} Evaluation:")
    print(f"R²: {r2:.3f}")
    print(f"RMSE: {rmse:.3f}")
    return y_pred, r2, rmse

## Model Workflow (Pipeline)

The complete model development process follows a structured pipeline to ensure consistency, reproducibility, and clarity. Each stage in the workflow is modularized into independent functions that can be reused for different water quality parameters. This modular approach streamlines the process and makes the workflow easily adaptable to new datasets or parameters in the future.

The pipeline automates the sequence of steps — from data preparation to evaluation — for each target parameter. The same set of predictor variables is used, while the response variable changes for each of the three targets: *Total Alkalinity (TA)*, *Electrical Conductance (EC)*, and *Dissolved Reactive Phosphorus (DRP)*. By maintaining a consistent framework, comparisons across models remain fair and interpretable.

In [10]:
def run_pipeline(X, y, param_name="Parameter"):
    print(f"\n{'='*60}")
    print(f"Training Model for {param_name}")
    print(f"{'='*60}")
    
    # Split data
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    # Scale
    X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test)
    
    # Train
    model = train_model(X_train_scaled, y_train)
    
    # Evaluate (in-sample)
    y_train_pred, r2_train, rmse_train = evaluate_model(model, X_train_scaled, y_train, "Train")
    
    # Evaluate (out-sample)
    y_test_pred, r2_test, rmse_test = evaluate_model(model, X_test_scaled, y_test, "Test")
    
    # Return summary
    results = {
        "Parameter": param_name,
        "R2_Train": r2_train,
        "RMSE_Train": rmse_train,
        "R2_Test": r2_test,
        "RMSE_Test": rmse_test
    }
    return model, scaler, pd.DataFrame([results])

### Model Training and Evaluation for Each Parameter

In this step, we apply the complete modeling pipeline to each of the three selected water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. The input feature set (`X`) remains the same across all three models, while the target variable (`y`) changes for each parameter. 

For every parameter, the `run_pipeline()` function is executed, which handles data preprocessing, model training, and both in-sample and out-of-sample evaluation. This ensures a consistent workflow and allows for a fair comparison of model performance across different water quality indicators.

In [11]:
X = wq_data.drop(columns=['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus'])

y_TA = wq_data['Total Alkalinity']
y_EC = wq_data['Electrical Conductance']
y_DRP = wq_data['Dissolved Reactive Phosphorus']

model_TA, scaler_TA, results_TA = run_pipeline(X, y_TA, "Total Alkalinity")
model_EC, scaler_EC, results_EC = run_pipeline(X, y_EC, "Electrical Conductance")
model_DRP, scaler_DRP, results_DRP = run_pipeline(X, y_DRP, "Dissolved Reactive Phosphorus")

In [ ]:
model_TA, scaler_TA, results_TA = run_pipeline(X, y_TA, "TOTAL_ALKALINITY")
model_EC, scaler_EC, results_EC = run_pipeline(X, y_EC, "ELECTRICAL_CONDUCTANCE")
model_DRP, scaler_DRP, results_DRP = run_pipeline(X, y_DRP, "DISSOLVED_REACTIVE_PHOSPHORUS")

### Model Performance Summary

After training and evaluating the models for each water quality parameter, the individual performance metrics are combined into a single summary table. This table consolidates the R² and RMSE values for both in-sample and out-of-sample evaluations, enabling an easy comparison of model performance across Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. 

Such a summary provides a quick overview of how well each model captures the variability in each parameter and highlights any differences in predictive accuracy.

In [12]:
results_summary = pd.concat([results_TA, results_EC, results_DRP], ignore_index=True)
results_summary

## Submission

Once you are satisfied with your model’s performance, you can proceed to make predictions for unseen data. To do this, use your trained model to estimate the concentrations of the target water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus — for a set of test locations provided in the **Submission_template.csv** file. 

The predicted results can then be uploaded to the challenge platform for evaluation.

MY CODE: READ SUBMISSION FROM TABLES

In [ ]:
sub_feat = session.table("FEATURES.FEATURE_MART_SUBMISSION_IMPUTED").to_pandas()
sub_template = session.table("RAW.SUBMISSION_TEMPLATE").to_pandas()

display(sub_feat.head())
display(sub_template.head())

In [ ]:
test_file = pd.read_csv("submission_template.csv")
display(test_file.head(5))

In [ ]:
landsat_val_features = pd.read_csv("landsat_features_validation.csv")
display(landsat_val_features.head(5))

In [ ]:
Terraclimate_val_df = pd.read_csv("terraclimate_features_validation.csv")
display(Terraclimate_val_df.head(5))

Similarly, participants can use the **Landsat** and **TerraClimate** data extraction demonstration notebooks to produce feature CSVs for their **validation** data. For convenience, we have already computed and saved example validation outputs as `landsat_features_val_V3.csv` and `Terraclimate_val_df_v3.csv`. 

Participants should save their own extracted files in the same format and column schema; doing so will allow this benchmark notebook to load the validation features directly and run smoothly.

In [16]:
#Consolidate all the extracted bands and features in a single dataframe
val_data = pd.DataFrame({
    'Longitude': landsat_val_features['Longitude'].values,
    'Latitude': landsat_val_features['Latitude'].values,
    'Sample Date': landsat_val_features['Sample Date'].values,
    'nir': landsat_val_features['nir'].values,
    'green': landsat_val_features['green'].values,
    'swir16': landsat_val_features['swir16'].values,
    'swir22': landsat_val_features['swir22'].values,
    'NDMI': landsat_val_features['NDMI'].values,
    'MNDWI': landsat_val_features['MNDWI'].values,
    'pet': Terraclimate_val_df['pet'].values,
})

In [17]:
# Impute the missing values
val_data = val_data.fillna(val_data.median(numeric_only=True))

In [ ]:
# Extracting specific columns (swir22, NDMI, MNDWI, pet) from the validation dataset
submission_val_data=val_data.loc[:,['swir22','NDMI','MNDWI','pet']]
display(submission_val_data.head())

MY CODE: PREDICTION FEATURES

In [ ]:
# Features para predecir
X_sub = sub_feat[FEATURES]

In [19]:
submission_val_data.shape

MY CODE: PREDICTION

In [ ]:
X_sub_scaled_TA = scaler_TA.transform(X_sub)
pred_TA_submission = model_TA.predict(X_sub_scaled_TA)

X_sub_scaled_EC = scaler_EC.transform(X_sub)
pred_EC_submission = model_EC.predict(X_sub_scaled_EC)

X_sub_scaled_DRP = scaler_DRP.transform(X_sub)
pred_DRP_submission = model_DRP.predict(X_sub_scaled_DRP)

In [20]:
# --- Predicting for Total Alkalinity ---
X_sub_scaled_TA = scaler_TA.transform(submission_val_data)
pred_TA_submission = model_TA.predict(X_sub_scaled_TA)

# --- Predicting for Electrical Conductance ---
X_sub_scaled_EC = scaler_EC.transform(submission_val_data)
pred_EC_submission = model_EC.predict(X_sub_scaled_EC)

# --- Predicting for Dissolved Reactive Phosphorus ---
X_sub_scaled_DRP = scaler_DRP.transform(submission_val_data)
pred_DRP_submission = model_DRP.predict(X_sub_scaled_DRP)

MY CODE: SUBMISSION DF

In [ ]:
import hashlib
import pandas as pd

def make_point_id(lat, lon, sample_date):
    lat_r = round(float(lat), 5)
    lon_r = round(float(lon), 5)
    d = pd.to_datetime(sample_date).date()
    raw = f"{lat_r}|{lon_r}|{d}"
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()

sub_template = sub_template.copy()
sub_template["POINT_ID"] = sub_template.apply(
    lambda r: make_point_id(r["LATITUDE"], r["LONGITUDE"], r["SAMPLE_DATE"]),
    axis=1
)

preds = pd.DataFrame({
    "POINT_ID": sub_feat["POINT_ID"],
    "TOTAL_ALKALINITY": pred_TA_submission,
    "ELECTRICAL_CONDUCTANCE": pred_EC_submission,
    "DISSOLVED_REACTIVE_PHOSPHORUS": pred_DRP_submission
})

final = sub_template.merge(preds, on="POINT_ID", how="left")

submission_df = pd.DataFrame({
    "Longitude": final["LONGITUDE"].values,
    "Latitude": final["LATITUDE"].values,
    "Sample Date": final["SAMPLE_DATE"].values,
    "Total Alkalinity": final["TOTAL_ALKALINITY"].values,
    "Electrical Conductance": final["ELECTRICAL_CONDUCTANCE"].values,
    "Dissolved Reactive Phosphorus": final["DISSOLVED_REACTIVE_PHOSPHORUS"].values
})

display(submission_df.head())

In [21]:
submission_df = pd.DataFrame({
    'Longitude': test_file['Longitude'].values,
    'Latitude': test_file['Latitude'].values,
    'Sample Date': test_file['Sample Date'].values,
    'Total Alkalinity': pred_TA_submission,
    'Electrical Conductance': pred_EC_submission,
    'Dissolved Reactive Phosphorus': pred_DRP_submission
})

In [22]:
#Displaying the sample submission dataframe
display(submission_df.head())

In [23]:
#Dumping the predictions into a csv file.
submission_df.to_csv("/tmp/submission.csv",index = False)

In [ ]:
session.sql(f"""
    PUT file:///tmp/submission.csv
    snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

### Upload submission file on platform

Upload the `submission.csv` file on the challenge platform to generate your score on the leaderboard.

## Conclusion

Now that you have learned a basic approach to model training, it’s time to explore your own techniques and ideas! Feel free to modify any of the functions presented in this notebook to experiment with alternative preprocessing steps, feature engineering strategies, or machine learning algorithms. 

We look forward to seeing your enhanced model and the insights you uncover. Best of luck with the challenge!